In [12]:
import openpyxl
import pandas as pd
import re
import numpy as np

In [13]:
data = pd.read_excel("tuition_data.xlsx", )  
data


,University,Campus,Faculty,Program,Tuition Fee
0,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...
1,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...
2,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมหุ่นยนต์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมหุ...,ดูรายละเอียดเพิ่มเติม https://www.chula.ac.th/...
3,มหาวิทยาลัยเกษตรศาสตร์,บางเขน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,0
4,มหาวิทยาลัยเกษตรศาสตร์,กำแพงแสน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,"156,000"
...,...,...,...,...,...
57,มหาวิทยาลัยรังสิต,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมคอมพิวเตอร์,"ค่าเล่าเรียนตลอดหลักสูตร 396,800"
58,มหาวิทยาลัยธุรกิจบัณฑิตย์,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ค่าเล่าเรียนตลอดหลักสูตร 337000 บาท
59,มหาวิทยาลัยเกษมบัณฑิต,วิทยาเขตร่มเกล้า,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมคอมพิวเตอร์,"282,000 บาท ตลอดหลักสูตร"
60,มหาวิทยาลัยสยาม,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,"ประมาณการอัตราค่าเล่าเรียน ตลอดหลักสูตร 303,50..."


In [14]:
filtered_data = data[
    (data['Faculty'].isin(['วิศวกรรมคอมพิวเตอร์', 'วิศวกรรมปัญญาประดิษฐ์', 'วิศวกรรมหุ่นยนต์'])) |
    (data['Program'].isin([
        'วิศวกรรมคอมพิวเตอร์',
        'วิศวกรรมปัญญาประดิษฐ์',
        'คอมพิวเตอร์',
        'ปัญญาประดิษฐ์',
    ]))
]
row_count = len(filtered_data)
filtered_data = filtered_data.reset_index(drop=True)
print("Number of rows:", row_count)
filtered_data

Number of rows: 60


,University,Campus,Faculty,Program,Tuition Fee
0,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...
1,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...
2,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมหุ่นยนต์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมหุ...,ดูรายละเอียดเพิ่มเติม https://www.chula.ac.th/...
3,มหาวิทยาลัยเกษตรศาสตร์,บางเขน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,0
4,มหาวิทยาลัยเกษตรศาสตร์,กำแพงแสน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,"156,000"
5,มหาวิทยาลัยเกษตรศาสตร์,ศรีราชา,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,"232,800"
6,มหาวิทยาลัยเกษตรศาสตร์,เฉลิมพระเกียรติ จ.สกลนคร,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,0
7,มหาวิทยาลัยขอนแก่น,ขอนแก่น,วิศวกรรมคอมพิวเตอร์,วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...,ค่าธรรมเนียมการศึกษา (ชาวไทย) 20000 บาท/เทอม
8,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมคอมพิวเตอร์,"184,000"
9,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมระบบสารสนเทศและเครือข่าย,"400,000"


In [15]:
import re
import pandas as pd

def classify_and_extract_fee(fee_text):
    if pd.isna(fee_text) or str(fee_text).strip() in ['0', '-', '_', '']:
        return ('ไม่ระบุ', None)

    fee_text = str(fee_text)

    if 'http' in fee_text or '.pdf' in fee_text:
        return ('ลิงก์', None)

    # ดึงตัวเลขทั้งหมด แล้วแปลงเป็น int อย่างถูกต้อง
    number_strings = re.findall(r'\d[\d,]*', fee_text)
    numbers = []
    for num_str in number_strings:
        try:
            num = int(num_str.replace(',', ''))
            numbers.append(num)
        except ValueError:
            continue

    # คำที่สื่อว่าเป็นค่าตลอดหลักสูตร
    if any(kw in fee_text for kw in ['ตลอดหลักสูตร', 'รวม', 'เรียน 8 ภาค', 'เรียน 6 ภาค']):
        return ('ค่าเรียนตลอดหลักสูตร', max(numbers) if numbers else None)

    # คำที่บ่งชี้ว่าเป็นรายภาคเรียน
    if any(kw in fee_text for kw in ['ต่อภาค', 'ภาคเรียนละ', '/เทอม', 'ต่อเทอม', 'ภาคการศึกษาละ', 'ภาคการศึกษาแรก']):
        return ('ค่าเทอม', max(numbers) if numbers else None)

    # ถ้าไม่มีคำบ่งชี้ แต่มีตัวเลขเดียว และมากกว่า 100,000 → ค่าตลอดหลักสูตร
    if numbers:
        max_num = max(numbers)
        if max_num > 100000:
            return ('ค่าเรียนตลอดหลักสูตร', max_num)
        else:
            return ('ค่าเทอม', max_num)

    return ('ไม่ระบุ', None)


In [16]:
filtered_data[['Tuition Category', 'Confirm Tution Fee']] = filtered_data['Tuition Fee'].apply(
    lambda x: pd.Series(classify_and_extract_fee(str(x)))
)

In [17]:
filtered_data

,University,Campus,Faculty,Program,Tuition Fee,Tuition Category,Confirm Tution Fee
0,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,ลิงก์,NaN
1,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,ลิงก์,NaN
2,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมหุ่นยนต์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมหุ...,ดูรายละเอียดเพิ่มเติม https://www.chula.ac.th/...,ลิงก์,NaN
3,มหาวิทยาลัยเกษตรศาสตร์,บางเขน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,0,ไม่ระบุ,NaN
4,มหาวิทยาลัยเกษตรศาสตร์,กำแพงแสน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,"156,000",ค่าเรียนตลอดหลักสูตร,156000.0
5,มหาวิทยาลัยเกษตรศาสตร์,ศรีราชา,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,"232,800",ค่าเรียนตลอดหลักสูตร,232800.0
6,มหาวิทยาลัยเกษตรศาสตร์,เฉลิมพระเกียรติ จ.สกลนคร,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,0,ไม่ระบุ,NaN
7,มหาวิทยาลัยขอนแก่น,ขอนแก่น,วิศวกรรมคอมพิวเตอร์,วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...,ค่าธรรมเนียมการศึกษา (ชาวไทย) 20000 บาท/เทอม,ค่าเทอม,20000.0
8,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมคอมพิวเตอร์,"184,000",ค่าเรียนตลอดหลักสูตร,184000.0
9,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมระบบสารสนเทศและเครือข่าย,"400,000",ค่าเรียนตลอดหลักสูตร,400000.0


In [18]:
link_count = (filtered_data['Tuition Category'] == 'ลิงก์').sum()
zero_count = (filtered_data['Tuition Fee'].astype(str).str.strip().isin(['0', '-', '_', ''])).sum()
print(f"Link count: {link_count}")
print(f"Zero or '-' count: {zero_count}")
link_rows = filtered_data.index[filtered_data['Tuition Category'] == 'ลิงก์'].tolist()
zero_rows = filtered_data.index[filtered_data['Tuition Fee'].astype(str).str.strip().isin(['0', '-', '_', ''])].tolist()
print(f"Row indices with 'ลิงก์': {link_rows}")
print(f"Row indices with '0', '-', or '_': {zero_rows}")

Link count: 8
Zero or '-' count: 4
Row indices with 'ลิงก์': [0, 1, 2, 18, 19, 20, 34, 40]
Row indices with '0', '-', or '_': [3, 6, 52, 54]


In [19]:
filtered_data.to_excel("filtered_tuition_data.xlsx", index=False)

In [20]:
filtered_data.loc[[0, 1, 2, 3, 6, 18, 19, 20, 34, 40, 52, 54], 'Tuition Category'] = 'ค่าเทอม'
filtered_data.loc[[0, 1, 2], 'Confirm Tution Fee'] = 25500
filtered_data.loc[[3], 'Confirm Tution Fee'] = 17300
filtered_data.loc[[6], 'Confirm Tution Fee'] = 15000
filtered_data.loc[[18], 'Confirm Tution Fee'] = 70000
filtered_data.loc[[19], 'Confirm Tution Fee'] = 30000
filtered_data.loc[[20], 'Confirm Tution Fee'] = 25000
filtered_data.loc[[34], 'Confirm Tution Fee'] = 21000
filtered_data.loc[[40], 'Confirm Tution Fee'] = 18500
filtered_data.loc[[52], 'Confirm Tution Fee'] = 40200
filtered_data.loc[[54], 'Confirm Tution Fee'] = 45825

filtered_data.loc[[3], 'Tuition Fee'] = 17300
filtered_data.loc[[6], 'Tuition Fee'] = 15000
filtered_data.loc[[52], 'Tuition Fee'] = 40200
filtered_data.loc[[54], 'Tuition Fee'] = 45825

In [21]:
filtered_data

,University,Campus,Faculty,Program,Tuition Fee,Tuition Category,Confirm Tution Fee
0,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,ค่าเทอม,25500.0
1,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,ภาคการศึกษาต้นและภาคการศึกษาปลาย ภาคการศึกษาละ...,ค่าเทอม,25500.0
2,จุฬาลงกรณ์มหาวิทยาลัย,วิทยาเขตหลัก,วิศวกรรมหุ่นยนต์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมหุ...,ดูรายละเอียดเพิ่มเติม https://www.chula.ac.th/...,ค่าเทอม,25500.0
3,มหาวิทยาลัยเกษตรศาสตร์,บางเขน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,17300,ค่าเทอม,17300.0
4,มหาวิทยาลัยเกษตรศาสตร์,กำแพงแสน,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,"156,000",ค่าเรียนตลอดหลักสูตร,156000.0
5,มหาวิทยาลัยเกษตรศาสตร์,ศรีราชา,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,"232,800",ค่าเรียนตลอดหลักสูตร,232800.0
6,มหาวิทยาลัยเกษตรศาสตร์,เฉลิมพระเกียรติ จ.สกลนคร,วิศวกรรมคอมพิวเตอร์,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์,15000,ค่าเทอม,15000.0
7,มหาวิทยาลัยขอนแก่น,ขอนแก่น,วิศวกรรมคอมพิวเตอร์,วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...,ค่าธรรมเนียมการศึกษา (ชาวไทย) 20000 บาท/เทอม,ค่าเทอม,20000.0
8,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมคอมพิวเตอร์,"184,000",ค่าเรียนตลอดหลักสูตร,184000.0
9,มหาวิทยาลัยเชียงใหม่,วิทยาเขตหลัก,วิศวกรรมคอมพิวเตอร์,วศ.บ.วิศวกรรมระบบสารสนเทศและเครือข่าย,"400,000",ค่าเรียนตลอดหลักสูตร,400000.0


In [22]:
filtered_data.to_excel("cleaned_tuition_data.xlsx", index=False)